In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from zipfile import ZipFile
import gzip
import concurrent
import zlib
import gc

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
class DatasetLoader:
    def __init__(self, file, csv, index_col=False, file_type='zip', lines=-1, max_blocks=-1):
        self.file = file
        self.csv = csv
        self.index_col = index_col
        self.file_type = file_type
        self.lines = lines
        self.max_blocks = max_blocks
        self.eof = False
        self.encoding = 'utf_8'
        self.prefix = 'drive/MyDrive/data/'
        #self.lock = Lock()

    def decompress_data(self):
        block = 0

        if self.file_type == 'zip':
            with ZipFile(f'{self.self.prefix}{self.file}.zip') as archive:
                file = archive.open(f'{self.csv}.csv')
                df = pd.read_csv(file, index_col=self.index_col)
                file.close()
            return df
        elif self.file_type == 'gz':
            with gzip.open(f'{self.prefix}{self.file}.csv.gz') as archive:
                df = pd.read_csv(archive, index_col=self.index_col)
            return df
        else:
            return None

    def decompress_stream(self):
        if self.file_type == 'gz':
            self.archive = gzip.GzipFile(f'{self.prefix}{self.file}.csv.gz', mode='rb')

            line_str = self.archive.readline().decode(self.encoding).replace('\n', '')
            self.headers = np.array(line_str.split(','), dtype=np.str_)

            block = 0
            while self.eof == False:
                #print('Loading block ' + str(block))

                with concurrent.futures.ThreadPoolExecutor(1) as executor:
                    fragment = executor.submit(self.process_fragment).result()

                block += 1
                if block == self.max_blocks:
                    self.eof = True

                yield fragment


    def process_fragment(self):
        #df_data = np.empty(shape=(self.lines, self.headers.size), dtype='O')
        df_data = []
        for i in range(0, self.lines):
            try:
                line_str = self.archive.readline().decode(self.encoding).replace('\n', '')

                if line_str == '':
                    if i == 0:
                        self.eof = True
                else:
                    input_data = np.array(line_str.split(','), dtype=np.str_)

                    if input_data.size < self.headers.size:
                        input_data = np.pad(input_data, (self.headers.size - input_data.size, 0), 'constant', constant_values='0')

                    if input_data.size == self.headers.size:
                        df_data.append(input_data)

                    #if i % 100 == 0:
                    #    print(i)
                    #    print(df_data[0])

            except zlib.error:
                #print('zlib error')
                pass

        df = pd.DataFrame(df_data, columns=self.headers)
        return df

zips = ['google_play', 'google_play_2016', 'steam']
csvs = [
    [
        'Google-Playstore'
    ],
    [
        'latest-with-added-date'
    ],
    [
        'steam',
        'steam_description_data',
        'steam_media_data',
        'steam_requirements_data',
        'steam_support_info',
        'steamspy_tag_data'
    ]
]

In [8]:
dl1 = DatasetLoader(zips[0],csvs[0][0],file_type='zip')
gp21_df = dl1.decompress_data()

In [67]:
def find_company_name(pkg_name):
    common = ['com', 'android', 'theme', 'app', 'air', 'apps', 'weather',
              'mobile', 'game', 'book', 'games', 'free', 'news', 'radio',
              'livewallpaper', 'photo', 'launcher', 'br', 'hr', 'co',
              'wallpaper', 'wallpapers' 'launcher', 'org', 'kbf', 'de', 'net',
              'ru', 'skin', 'keyboard', 'widget', 'jp', 'dev', 'live', 'music',
              'www', 'fr']

    pkg_names = pkg_name.split('.')

    if pkg_names[-2] in common:
        try:
            if pkg_names[-3] in common:
                if pkg_names[-1] in common:
                    return ''
                else:
                    return pkg_names[-1]
            else:
                return pkg_names[-3]
        except IndexError:
            if pkg_names[-1] in common:
                return ''
            else:
                return pkg_names[-1]
    else:
        return pkg_names[-2]

companies = gp16_df['pkg_name'].apply(find_company_name)

In [68]:
companies.value_counts()[0:30]

gokeyboard           7499
jfox                 3617
komakuro             2489
sharefaith           2258
mobincube            1869
touchpal             1813
magzter              1794
thechurchapp         1661
Tobit                1554
crowdcompass         1487
ionicframework       1335
goodbarber           1323
fitnessmobileapps    1302
mobomap               972
doubledutch           922
launcherex            878
wallpapers            813
fitivity              761
ays                   760
bestpuzzle            735
appbuilder            699
an                    682
gosms                 669
rxwikiplus            668
droidguide            658
ezweather             656
smartinputv5          643
attendify             629
skoolbag              626
client                622
Name: pkg_name, dtype: int64

In [28]:
gp16_df

,dex_date,pkg_name,vt_detection,vt_scan_date,added
0,2015-09-02 10:34:48,kr.ac.snjc.library,0,2016-03-17 04:48:32,2016-03-01 17:48:50.244774
1,2016-08-17 15:42:52,com.appautomatic.ankulua.trial,0,2016-11-22 17:06:20,2016-08-17 14:21:07.425706
2,2016-01-19 13:28:36,com.kbf.app27730661,0,2016-03-23 15:13:09,2016-03-01 21:43:50.759666
3,2016-01-02 23:48:34,com.bz.solo.theme.gray.dim,0,2016-02-21 08:32:22,2016-03-21 04:31:28.320733
4,2015-11-04 19:52:36,cn.dreamjet.newplanet.android,1,2016-03-15 03:05:28,2016-03-01 05:15:47.303595
...,...,...,...,...,...
1003769,2016-06-10 01:05:24,com.noikeo.candypuzzlelegend,0,2016-06-27 00:43:42,2016-06-13 13:54:48.481478
1003770,2014-02-19 00:18:12,apps.easylife.translator,0,2014-02-23 11:19:03,2014-02-19 07:06:11.542412
1003771,2016-05-26 19:07:48,com.vista.vistaapp,0,2016-06-06 03:33:30,2016-06-07 10:02:39.017294
1003772,2014-04-11 10:08:18,andromeda.galaxy.country.radio.station.de,0,2014-05-17 08:16:47,2014-05-13 07:00:04.72399


In [11]:
gp21_df

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312939,大俠客—熱血歸來,com.rxsj.ssjj,Role Playing,4.3,16775.0,"100,000+",100000.0,337109,True,0.0,...,http://www.4399sy.com.hk/,ssjjcomhk@gmail.com,NaN,"Jun 01, 2021",Teen,http://a.4399sy.com.hk/user/aggreement,False,False,False,2021-06-16 12:59:18
2312940,ORU Online,com.threedream.oruonline,Education,0.0,0.0,100+,100.0,430,True,0.0,...,http://www.oru.edu/,3DreamDeveloper@gmail.com,"Jan 17, 2018","Feb 02, 2018",Everyone,http://www.oru.edu/about-oru/privacy-policy.php,False,False,False,2021-06-16 12:59:19
2312941,Data Structure,datastructure.appoworld.datastucture,Education,0.0,0.0,100+,100.0,202,True,0.0,...,NaN,appoworld.official@gmail.com,"Aug 19, 2018","Aug 19, 2018",Everyone,https://appoworld.000webhostapp.com/datastruct...,False,False,False,2021-06-16 12:59:19
2312942,Devi Suktam,ishan.devi.suktam,Music & Audio,3.5,8.0,"1,000+",1000.0,2635,True,0.0,...,https://a70f78905.app-ads-txt.com,ruchisono@gmail.com,"Aug 1, 2016","May 05, 2021",Everyone,https://docs.google.com/document/d/1x-9reZuLRX...,True,False,False,2021-06-16 12:59:19


In [12]:
gp16_df['pkg_name'].value_counts()

"wp.wpbeta"                                    69
"com.imo.android.imoimbeta"                    64
"org.mozilla.firefox_beta"                     51
"summersedge23.com.circuitbreaker.lite"        49
"it.webxperience"                              45
                                               ..
"jp.co.solcreo.quiz0005"                        1
"com.legendphoto.seim2015sonular"               1
"com.ramadan.hadirli"                           1
"handasoft.mobile.fortune.solo"                 1
"andromeda.galaxy.country.radio.station.de"     1
Name: pkg_name, Length: 734794, dtype: int64

In [13]:
gp21_df['App Id'].value_counts()

com.ishakwe.gakondo              1
co.tapcart.app.id_GQEQ9eXxJg     1
com.apocalipseescatologia.app    1
com.puzzlegame.wordconnect       1
rankr.io.srigitanjali            1
                                ..
com.arabeauty.ringtones2020      1
com.bibigame.linkhalo            1
com.ilmasoftkids.mfb_eng         1
com.app.colonybuddy              1
com.yyazilim.biliyormusun        1
Name: App Id, Length: 2312944, dtype: int64

"kr.ac.snjc.library"                           1
"ru.tirexdev.requesttool"                      1
"turto120914.cover1226n"                       1
"com.mish.ali.crm50only"                       1
"com.guidepal.venice"                          1
                                              ..
"com.appbuilder.u79833p151550"                 1
"com.instantencore.necmusic_5113936"           1
"com.tc.kpssnedir"                             1
"com.musik.ostanakmenteng"                     1
"andromeda.galaxy.country.radio.station.de"    1
Name: pkg_name, Length: 734794, dtype: int64

In [ ]:
gp21_df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


In [ ]:
gp21_df['Category'].value_counts()

In [ ]:
gaming_categories = [
    'Arcade',
    'Puzzle',
    'Casual',
    'Action',
    'Simulation',
    'Adventure',
    'Board',
    'Racing',
    'Role Playing',
    'Strategy',
    'Card'
]

In [ ]:
# Include only games
gp21_df = gp21_df[gp21_df['Category'].isin(gaming_categories)]

In [24]:
gp16_df['pkg_name'][300:400]

300    "com.bestringtonesapps.birdcallssoundsandringt...
301                    "com.alvagamer.waxingmodelsgames"
302                                "com.leu.houseapM103"
303                                "ieltsvocab.academic"
304                                     "com.bmw.remote"
                             ...                        
395                            "com.app.app0e87ec29c687"
396                           "com.prosperworks.android"
397                                "com.talentle.talent"
398                 "com.crazygun.reaper.snipershooting"
399                                     "zou.app.kankan"
Name: pkg_name, Length: 100, dtype: object

In [ ]:
steam_lost_df = pd.read_csv(f'{prefix}steam_lost.csv', encoding='unicode_escape')
steam_lost_df.set_index('AppID', inplace=True)

In [ ]:
steam_df.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
appid,,,,,,,,,,,,,,,,,
10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [ ]:
steam_lost_df.head()

,Owners,Name,Type,Changed,Kinguin,Achievements
AppID,,,,,,
43110,77.48%,Metro 2033,Purchase disabled,03/2021,15.89,48
35420,71.18%,Killing Floor Mod: Defence Alliance 2,Purchase disabled,09/2022,NaN,30
310560,68.04%,DiRT Rally,Purchase disabled,11/2022,NaN,170
43160,67.58%,Metro: Last Light Complete Edition,Delisted,05/2023,9.07,70
863550,67.35%,HITMAN? 2,Purchase disabled,01/2023,NaN,146


In [ ]:
steam_df.join(steam_lost_df, how='right')

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,...,average_playtime,median_playtime,owners,price,Owners,Name,Type,Changed,Kinguin,Achievements
AppID,,,,,,,,,,,,,,,,,,,,,
43110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,77.48%,Metro 2033,Purchase disabled,03/2021,15.89,48
35420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,71.18%,Killing Floor Mod: Defence Alliance 2,Purchase disabled,09/2022,NaN,30
310560,DiRT Rally,2015-12-07,1.0,Codemasters Racing Studio;Feral Interactive (L...,Codemasters;Feral Interactive (Linux);Feral In...,windows;mac;linux,0.0,Single-player;Multi-player;Steam Achievements;...,Racing;Simulation;Sports,Racing;Simulation;Driving,...,488.0,251.0,1000000-2000000,24.99,68.04%,DiRT Rally,Purchase disabled,11/2022,NaN,170
43160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,67.58%,Metro: Last Light Complete Edition,Delisted,05/2023,9.07,70
863550,HITMAN™ 2,2018-11-13,1.0,IO Interactive A/S,Warner Bros Interactive Entertainment,windows,0.0,Single-player;Online Multi-Player;Online Co-op...,Action,Stealth;Action;Assassin,...,636.0,514.0,5000000-10000000,44.99,67.35%,HITMAN? 2,Purchase disabled,01/2023,NaN,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.01%,F1 23 Beta,Test app,01/2023,NaN,?
2299850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.01%,ERA-????,Unreleased,02/2023,NaN,?
2351270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.01%,Team Disorder Hodon UD,Purchase disabled,06/2023,NaN,?


In [ ]:
no_of_steam_games = steam_df.shape[0]
no_of_lost_steam_games = steam_lost_df.shape[0]

funnel_data = dict(
    number=[no_of_steam_games, no_of_lost_steam_games],
    stage=['Current Steam Games', 'Lost Steam Games']
)
fig = px.funnel(funnel_data, x='number', y='stage')
fig.show()